In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np
import PIL

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### DATASET

In [11]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f'{self.img_dir}/{item["id"]}.png')
        h, w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w  # Adjust x coordinate
        kps[1::2] *= 224.0 / h  # Adjust y coordinate

        return img, kps

In [12]:
train_dataset = KeypointsDataset('data/images', 'data/data_train.json')
val_dataset = KeypointsDataset('data/images', 'data/data_val.json')

train_dataloader = DataLoader(dataset=train_dataset,batch_size=8, shuffle=True)
val_dataset = DataLoader(dataset=val_dataset, batch_size=8, shuffle=True)

# Create Model

In [13]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) #replace last layer where 14 are the points on the field, times 2 ,cuz x and y coordinate

model = model.to(device)

C:\Users\hnikolov\AppData\Local\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hnikolov\AppData\Local\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\hnikolov/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:17<00:00, 5.77MB/s]


### Train model

In [14]:
criteria = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

In [16]:
epochs = 20
for epoch in range(epochs):
    for i, (imgs, kps) in enumerate(train_dataloader):
        imgs = imgs.to(device)
        kps = kps.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(imgs)
        loss = criteria(outputs, kps)
        loss.backward()
        optimizer.step()
        
        if i % 10 ==0:
            print(f'Epoch {epoch}, iter {i}, loss: {loss.item()}')

Epoch 0, iter 0, loss: 15072.3486328125
Epoch 0, iter 10, loss: 14625.8427734375
Epoch 0, iter 20, loss: 14718.3349609375


KeyboardInterrupt: 